In [1]:
def calculate_system_specs(available_area: float, panel_area: float, panel_wattage: float, peak_sun_hours: float, cost_per_panel: float, installation_cost_per_panel: float, fixed_cost: float) -> list[float]:
    """

    :param available_area:
    :param panel_area:
    :param panel_wattage:
    :param peak_sun_hours:
    :param cost_per_panel:
    :param installation_cost_per_panel:
    :param fixed_cost:
    :return:
    """
    num_panels = available_area // panel_area
    total_cost = num_panels * (cost_per_panel + installation_cost_per_panel) + fixed_cost
    daily_kwh = num_panels * panel_wattage * peak_sun_hours / 1000
    annual_kwh = daily_kwh * 365
    return [num_panels, total_cost, daily_kwh, annual_kwh]

In [2]:
def calculate_yearly_production(initial_annual_kwh: float, degradation_rate: float, years: int) -> list[float]:
    """

    :param initial_annual_kwh:
    :param degradation_rate:
    :param years:
    :return:
    """
    output = []
    production = initial_annual_kwh
    degradation = 1
    for i in range(years):
        output.append(production * degradation)
        degradation -= degradation_rate
    return output

In [3]:
def calculate_cumulative_value(yearly_production: list[float], electricity_rate: float, rate_increase: float, total_cost: float) -> list[float]:
    """

    :param yearly_production:
    :param electricity_rate:
    :param rate_increase:
    :param total_cost:
    :return:
    """
    cumulative_savings = 0
    output = []
    for i in yearly_production:
        cumulative_savings += i * electricity_rate
        output.append(cumulative_savings - total_cost)
        electricity_rate *= 1 + rate_increase
    return output

In [4]:
def find_best_option(options: list[str], specs_list: list[list[float]], values_list: list[list[float]], priority: str) -> str:
    if priority == 'cost':
        output = options[0]
        lowest_cost = specs_list[0][1]
        for option, specs in zip(options, specs_list):
            if specs[1] < lowest_cost:
                lowest_cost = specs[1]
                output = option
        return output
    elif priority == 'production':
        output = options[0]
        highest_annual = specs_list[0][3]
        for option, specs in zip(options, specs_list):
            if specs[1] < highest_annual:
                highest_annual = specs[3]
                output = option
        return output
    else:
        output = options[0]
        most_value = values_list[0][-1]
        for option, values in zip(options, values_list):
            if values[-1] > most_value:
                most_value = values[-1]
                output = option
        return output

In [5]:
# Common parameters
area = 800
hours = 5.5
fixed = 3500
panel_installation = 140
rate = 0.14
r_increase = 0.035
lifetime = 25

In [6]:
def print_info(name):
    specs = calculate_system_specs(area, p_area, p_wattage, hours, panel_cost, panel_installation, fixed)
    num_panels, total_cost, daily_kwh, annual_kwh = specs
    yearly_production = calculate_yearly_production(annual_kwh, d_rate, lifetime)
    cumulative_value = calculate_cumulative_value(yearly_production, rate, r_increase, total_cost)
    payback_year = None
    for i, j in enumerate(cumulative_value):
        if j >= 0:
            payback_year = i + 1
            break
    total_production = sum(yearly_production)
    left_spacing = 30
    right_spacing = 9
    print(f'{name}\n'
          f'{'Number of panels':<{left_spacing}}{num_panels:>{right_spacing}.0f}\n'
          f'{'Total cost':<{left_spacing}}{f'${total_cost:,.0f}':>{right_spacing}}\n'
          f'{'Daily production (kwh)':<{left_spacing}}{f'{daily_kwh:.0f}':>{right_spacing}}\n'
          f'{'Year 1 production (kwh)':<{left_spacing}}{f'{yearly_production[0]:,.0f}':>{right_spacing}}\n'
          f'{'Final year production (kwh)':<{left_spacing}}{f'{yearly_production[-1]:,.0f}':>{right_spacing}}\n'
          f'{'Total energy production (kwh)':<{left_spacing}}{f'{total_production:,.0f}':>{right_spacing}}\n'
          f'{'Production decline':<{left_spacing}}{f'{(yearly_production[0] - yearly_production[-1])/yearly_production[0]*100:,.0f}%':>{right_spacing}}\n'
          f'{'Payback year':<{left_spacing}}{f'{payback_year}':>{right_spacing}}\n'
          f'{'Final net value':<{left_spacing}}{f'${cumulative_value[-1]:,.0f}':>{right_spacing}}\n')
    return specs, cumulative_value

In [7]:
name1 = 'Budget Panels'
p_area = 18.5
p_wattage = 310
panel_cost = 180
d_rate = 0.008
specs1, values1 = print_info(name1)

Budget Panels
Number of panels                     43
Total cost                      $17,260
Daily production (kwh)               73
Year 1 production (kwh)          26,760
Final year production (kwh)      21,622
Total energy production (kwh)   604,775
Production decline                  19%
Payback year                          5
Final net value                $112,590



In [8]:
name2 = 'Standard Panels'
p_area = 17.5
p_wattage = 370
panel_cost = 285
d_rate = 0.005
specs2, values2 = print_info(name2)

Standard Panels
Number of panels                     45
Total cost                      $22,625
Daily production (kwh)               92
Year 1 production (kwh)          33,425
Final year production (kwh)      29,414
Total energy production (kwh)   785,485
Production decline                  12%
Payback year                          5
Final net value                $147,094



In [9]:
name3 = 'Premium Panels'
p_area = 17
p_wattage = 420
panel_cost = 395
d_rate = 0.003
specs3, values3 = print_info(name3)

Premium Panels
Number of panels                     47
Total cost                      $28,645
Daily production (kwh)              109
Year 1 production (kwh)          39,628
Final year production (kwh)      36,775
Total energy production (kwh)   955,036
Production decline                   7%
Payback year                          5
Final net value                $178,521



In [10]:
best_cost = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'cost')
best_production = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'production')
best_value = find_best_option([name1, name2, name3], [specs1, specs2, specs3], [values1, values2, values3], 'value')
print('Best cost:', best_cost)
print('Best production:', best_production)
print('Best value:', best_value)

Best cost: Budget Panels
Best production: Premium Panels
Best value: Premium Panels


Question 1
Option 3 shows the smallest production decline (7%)